# Как распарсить вал и трейн сеты

Можете парсить как нравится, я просто подумала так будет удобнее

In [7]:
import json
from func_to_call import parse_all_data, parse_data_with_time


data_v1 = parse_all_data('datasets/val_set.json')

data_v2 = parse_data_with_time('datasets/val_set.json')

with open('parsed_tuning.json', 'w', encoding='utf-8') as f:
    json.dump(data_v1, f, ensure_ascii=False)

with open('parsed_dash.json', 'w', encoding='utf-8') as f:
    json.dump(data_v2, f, ensure_ascii=False)

In [8]:
import json

with open('parsed_tuning.json', 'r', encoding='utf-8') as f:
    training_data = json.load(f)

In [9]:
formatted_data = []

for item in training_data:
    contexts = "\n".join([ctx['text'] for ctx in item['contexts']])
    base_input = f"Вопрос: {item['user_question']}\nКонтекст: {contexts}"

    if item['winner'] == 'Saiga':
        formatted_data.append({
            "input": base_input,
            "output": item['saiga_answer'],
            "source": "saiga",
            "rating": "good" if item['winner'] in ['Saiga', 'Оба хорошо'] else "bad"
        })
        
    elif item['winner'] == 'GigaChat':
        formatted_data.append({
            "input": base_input,
            "output": item['giga_answer'],
            "source": "giga",
            "rating": "good" if item['winner'] in ['GigaChat', 'Оба хорошо'] else "bad"
        })
        
    elif item['winner'] == 'Оба хорошо':
        formatted_data.extend([
            {
                "input": base_input,
                "output": item['saiga_answer'],
                "source": "saiga",
                "rating": "good"
            },
            {
                "input": base_input,
                "output": item['giga_answer'],
                "source": "giga",
                "rating": "good"
            }
        ])
        
    elif item['winner'] == 'Оба плохо':
        formatted_data.extend([
            {
                "input": base_input,
                "output": item['saiga_answer'],
                "source": "saiga",
                "rating": "bad"
            },
            {
                "input": base_input,
                "output": item['giga_answer'],
                "source": "giga",
                "rating": "bad"
            }
        ])
        
    else:
        formatted_data.append({
            "input": base_input,
            "output": item['saiga_answer'],
            "source": "unknown",
            "rating": "neutral"
        })

In [10]:
formatted_data[0]

{'input': 'Вопрос: Чем заняться вечером, если не хочется сидеть дома?\nКонтекст: ающихся) В соответствии с частью 4 статьи 34 Федерального закона от 29.12.2012 №273-ФЗ «Об образовании в Российской Федерации» НИУ ВШЭ обеспечивает обучающимся реализацию их права на посещение по своему выбору мероприятий, которые проводятся в университете и не предусмотрены учебным планом. К числу мероприятий, не предусмотренных учебным планом, относятся праздники, конкурсы, тематические вечера, фестивали, спортивные соревнования, выставки и другие мероприятия, порядок проведения которых не установлен иными локальными нормативными актами НИУ ВШЭ. Мероприятия, не предусмотренные учебным планом, проводятся в целях: 1.4.1. личностного развития обучающихся, формирования профессиональной культуры будущих специалистов, их гражданской позиции, мировоззрения и социальной активности; 1.4.2. реализации гражданско-патриотического, эстетического, духовного физического воспитания и профессиональной ориентации обучающи

# Формат того как должен выглядеть датасет чтобы метрики посчитать

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_json('parsed_dash.json')

dataset = df.copy()

df = df.rename(columns={'user_question': 'question'})

df['ground_truth'] = np.where(df['winner'].str.upper() == 'GIGA', df['giga_answer'], df['saiga_answer'])
df['answer'] = np.where(df['winner'].str.upper() == 'GIGA', df['saiga_answer'], df['giga_answer'])

df['contexts'] = df['contexts'].apply(lambda x: x if len(x) > 0 else [""])

df = df[['question', 'answer', 'ground_truth', 'contexts']]

df.head()

from metrics import ValidatorSimple

vs = ValidatorSimple(neural=True)
vs.validate_rag(df)

dataset.head()

score_sample: 174it [02:39,  1.09it/s]


,selected_role,campus,education_level,question_category,user_question,user_filters,question_filters,saiga_answer,giga_answer,winner,comment,contexts,response_time,refined_response_time,refined_question,refined_answer,refined_contexts
0,Студент,Нижний Новгород,Бакалавриат,Внеучебка,"Чем заняться вечером, если не хочется сидеть д...","[Нижний Новгород, бакалавриат]",[Внеучебка],"Я понял ваш запрос о том, что вы хотите узнать...","Что-то в вашем вопросе меня смущает. Может, по...",Saiga,None,[{'text': 'ающихся) В соответствии с частью 4 ...,1.814384,1.738566,NaN,NaN,NaN
1,Студент,Нижний Новгород,Бакалавриат,Военка,На каком курсе начинается военная кафедра и в ...,"[Нижний Новгород, бакалавриат]",[ВУЦ],Я нашла информацию о начале военной кафедры и ...,Информация о начале курса военной кафедры и ко...,Оба плохо,None,[{'text': 'в ходе которого студенты приводятся...,4.327234,NaN,NaN,NaN,NaN
2,Студент,Нижний Новгород,Бакалавриат,Учеба,Когда начинается неделя зимней сессии?,"[Нижний Новгород, бакалавриат]","[Учебный процесс, Практическая подготовка, ГИА...",Я понял ваш вопрос о начале зимней сессии. Сог...,Информация о начале зимней сессии не представл...,Оба плохо,Галлюцинация,[{'text': 'Майнор Майнор (Minor) Зимняя кампан...,3.580883,4.030733,Когда начинается неделя зимней сессии?Скажи ко...,Зимняя кампания по переводам между майнорами н...,[{'text': 'Майнор Майнор (Minor) Зимняя кампан...
3,Студент,Нижний Новгород,Бакалавриат,Социальные вопросы,Как я могу получить бесплатную консультацию у ...,"[Нижний Новгород, бакалавриат]","[Социальные вопросы, Безопасность, Общежития]","Я нашла информацию о том, что для получения бе...",Для получения бесплатной консультации у психол...,Оба хорошо,None,[{'text': 'Психологическая помощь и телефоны г...,1.877145,4.089629,Как получить справку об обучении в НИУ ВШЭ?Где...,Справки об обучении в НИУ ВШЭ можно заказать ч...,[{'text': 'в НИУ ВШЭ: справка о подтверждении ...
4,Студент,Нижний Новгород,Бакалавриат,Финансовые вопросы,"Что будет, если не оплатить обучение вовремя?","[Нижний Новгород, бакалавриат]",[Деньги],Я понял ваш страх по поводу опоздания с оплато...,"Если не оплатить обучение вовремя, это может п...",Оба хорошо,None,[{'text': 'договором. Оплата образовательных у...,4.603413,4.030733,Когда начинается неделя зимней сессии?Скажи ко...,Зимняя кампания по переводам между майнорами н...,[{'text': 'Майнор Майнор (Minor) Зимняя кампан...


In [ ]:
giga_count = (dataset['winner'].str.upper() == 'GIGA').sum()
saiga_count = (dataset['winner'].str.upper() == 'SAIGA').sum()

print("GIGA: ", giga_count)
print("SAIGA: ", saiga_count)

GIGA:  0
SAIGA:  37


В файле с метриками prepocess_calculate/metrics.py указано описание каждой метрики
